In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
#r "nuget: System.ClientModel, 1.6.1.0"

Installed Packages System.ClientModel, 1.6.1

In [3]:
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"

Installed Packages Azure.Identity, 1.15.0 System.Linq.Async, 6.0.3

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.Workflows\bin\Debug\net9.0\Microsoft.Agents.Workflows.dll"

In [5]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0\Microsoft.Agents.AI.OpenAI.dll"

In [6]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [9]:
using System;
using System.ComponentModel;
using System.ClientModel;
using OpenAI;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.Workflows;

In [10]:
 using DotNetEnv;

In [11]:
Env.Load("../../../.env");

In [12]:

var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id =  "gpt-4o";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

var imgPath ="../imgs/home.png";

In [13]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint = new Uri(github_endpoint)
};

In [14]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [15]:
const string SalesAgentName = "Sales-Agent";
const string SalesAgentInstructions = "You are my furniture sales consultant, you can find different furniture elements from the pictures and give me a purchase suggestion";

In [16]:
const string PriceAgentName = "Price-Agent";
const string PriceAgentInstructions = @"You are a furniture pricing specialist and budget consultant. Your responsibilities include:
        1. Analyze furniture items and provide realistic price ranges based on quality, brand, and market standards
        2. Break down pricing by individual furniture pieces
        3. Provide budget-friendly alternatives and premium options
        4. Consider different price tiers (budget, mid-range, premium)
        5. Include estimated total costs for room setups
        6. Suggest where to find the best deals and shopping recommendations
        7. Factor in additional costs like delivery, assembly, and accessories
        8. Provide seasonal pricing insights and best times to buy
        Always format your response with clear price breakdowns and explanations for the pricing rationale.";

In [17]:
const string QuoteAgentName = "Quote-Agent";
const string QuoteAgentInstructions = @"You are a assistant that create a quote for furniture purchase.
        1. Create a well-structured quote document that includes:
        2. A title page with the document title, date, and client name
        3. An introduction summarizing the purpose of the document
        4. A summary section with total estimated costs and recommendations
        5. Use clear headings, bullet points, and tables for easy readability
        6. All quotes are presented in markdown form";

In [18]:
using System.IO;

async Task<byte[]> OpenImageBytesAsync(string path)
{
	return await File.ReadAllBytesAsync(path);
}

var imageBytes = await OpenImageBytesAsync(imgPath);

In [19]:
imageBytes

[ 137, 80, 78, 71, 13, 10, 26, 10, 0, 0, 0, 13, 73, 72, 68, 82, 0, 0, 3, 142 ... (823194 more) ]

In [20]:
AIAgent salesagent = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:SalesAgentName,instructions:SalesAgentInstructions);
AIAgent priceagent  = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:PriceAgentName,instructions:PriceAgentInstructions);
AIAgent quoteagent = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:QuoteAgentName,instructions:QuoteAgentInstructions);

In [21]:
var workflow = new WorkflowBuilder(salesagent)
            .AddEdge(salesagent,priceagent)
            .AddEdge(priceagent, quoteagent)
            .Build();

In [22]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new DataContent(imageBytes, "image/png"),new TextContent("Please find the relevant furniture according to the image and give the corresponding price for each piece of furniture.Finally Output generates a quotation") 
]);

In [23]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, userMessage);

In [24]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));
string id="";
string messageData="";
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
    if (evt is AgentRunUpdateEvent executorComplete)
    {
        if(id=="")
        {
            id=executorComplete.ExecutorId;
        }
        if(id==executorComplete.ExecutorId)
        {
            messageData+=executorComplete.Data.ToString();
        }
        else
        {
            id=executorComplete.ExecutorId;
        }
        // Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
    }
}

Console.WriteLine(messageData);

Certainly! Below is a list of relevant furniture elements analyzed from the image along with estimated prices based on general market standards. Prices can vary depending on quality, brand, and retailer.

### Furniture List and Estimated Prices:

1. **Sofa: White 3-Seater with Cushions**
   - Description: A minimalist white fabric sofa paired with soft blue and black throw pillows.
   - Estimated Price: $800

2. **Center Coffee Table**
   - Description: Oval-shaped modern wooden coffee table with light-colored wood and angled legs.
   - Estimated Price: $200

3. **TV Console Unit**
   - Description: A long, light wooden low-profile console with storage cabinets and open shelving.
   - Estimated Price: $400

4. **Armchair**
   - Description: Deep navy-blue, winged back chair with black slender legs, adding an accent.
   - Estimated Price: $300

5. **Floor Lamp with Decorative Base**
   - Description: A slim, metallic floor lamp with round glass ornaments below, ideal for modern decor.
 